In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

housing_data = pd.read_csv('kc_house_data.csv')
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

train_data = train_data.loc[:, ~train_data.columns.str.contains("^unnamed", case=False)]
test_data  = test_data.loc[:, ~test_data.columns.str.contains("^unnamed", case=False)]


X_train = train_data['sqft_living'].to_numpy().reshape(-1, 1)
Y_train = train_data['price']/1000

X_test = test_data['sqft_living'].to_numpy().reshape(-1, 1)
Y_test = test_data['price']/1000

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
X_train_gd = np.column_stack([np.ones(X_train_scaled.shape[0]), X_train_scaled])
X_test_gd  = np.column_stack([np.ones(X_test_scaled.shape[0]),  X_test_scaled])

def gradient_descent(X, y, alpha, num_iters, theta0=None):
    m, n = X.shape
    theta = np.zeros(n) if theta0 is None else theta0.copy()

    for _ in range(num_iters):
        preds = X @ theta              
        grad = (X.T @ (preds - y)) / m 
        theta = theta - alpha * grad

    return theta

def eval_model(X_train, Y_train, X_test, Y_test, theta):
    Ytrain_pred = X_train @ theta
    Ytest_pred = X_test @ theta

    return {
        "Train MSE": mean_squared_error(Y_train, Ytrain_pred),
        "Train R^2": r2_score(Y_train, Ytrain_pred),
        "Test MSE": mean_squared_error(Y_test, Ytest_pred),
        "Test R^2": r2_score(Y_test, Ytest_pred),
    }


In [12]:
alphas = [0.01, 0.1, 0.5]
checkpoints = [10, 50, 100]

rows = []

for alpha in alphas:
    theta = np.zeros(X_train_gd.shape[1])  
    prev_iter = 0

    for it in checkpoints:
        theta = gradient_descent(X_train_gd, Y_train, alpha, it - prev_iter, theta0=theta)
        prev_iter = it

        metrics = eval_model(X_train_gd, Y_train, X_test_gd, Y_test, theta)

        rows.append({
            "alpha": alpha,
            "iters": it,
            "theta_norm": float(np.linalg.norm(theta)),  # compact summary of theta
            "theta0": float(theta[0]),                   # intercept
            **metrics
        })

results_df = pd.DataFrame(rows)
print(results_df)


   alpha  iters  theta_norm      theta0      Train MSE  Train R^2  \
0   0.01     10   54.763387   49.760987  326238.379810  -1.833476   
1   0.01     50  226.225425  205.560702  178013.911210  -0.546103   
2   0.01    100  363.093179  329.926174  101895.706189   0.115006   
3   0.10     10  373.032303  338.957401   97827.252013   0.150342   
4   0.10     50  569.779663  517.732733   57956.238864   0.496633   
5   0.10    100  572.716179  520.401011   57947.526393   0.496709   
6   0.50     10  572.172084  519.906616   57947.838987   0.496706   
7   0.50     50  572.731392  520.414834   57947.526161   0.496709   
8   0.50    100  572.731392  520.414834   57947.526161   0.496709   

        Test MSE  Test R^2  
0  394527.864096 -1.366311  
1  230236.737185 -0.380921  
2  143850.402653  0.137210  
3  139124.701210  0.165554  
4   88707.201279  0.467949  
5   88576.607457  0.468733  
6   88599.424093  0.468596  
7   88575.978543  0.468736  
8   88575.978543  0.468736  
